# Transform points from pitch to texture and vice-versa
### Matrix P - Projection
The P matrix is a 3x4 matrix that given a 3D point in the world reference frame, it is projected into the 2D image in **texture coordinate** reference frame, i.e:

\begin{align}
p_w & = (x, y, z, 1) \\
p_p & = (a, b, c) = P * p_w \\
p_t & = (i,j) = (a/c, b/c)
\end{align}

**Texture coordinate range** is [0.0, 1.0]

<img src="pitch.png" style="width: 400px;">

### Matrix H - Homography
The H matrix is a 3x3 Matrix that transforms points from one plane to another. In our case from pitch to texture and from texture to pitch which is the inverse of the first (pitch to texture homography).

<img src="homo.png" style="width: 400px;">

\begin{align}
texture & = H * pitch \\
pitch & = inverse(H) * texture
\end{align}



In [162]:
# import what we need
import numpy as np
import math

In [163]:
def rodrigues(r):
    '''
    Rodrigues formula
    :param r: 1x3 array of rotations about x, y, and z
    :return: the 3x3 rotation matrix
    '''
    def S(n):
        Sn = np.array([
            [0.0, -n[2], n[1]],
            [n[2], 0.0, -n[0]],
            [-n[1], n[0], 0]])
        return Sn
    theta = np.linalg.norm(r)
    if theta > 1e-30:
        n = r/theta
        Sn = S(n)
        R = np.eye(3) + np.sin(theta) * Sn + (1.0 - np.cos(theta)) * np.dot(Sn, Sn)
    else:
        Sr = S(r)
        theta2 = theta ** 2.0
        R = np.eye(3) + (1.0 - theta2 / 6.0)*Sr + (0.5 - theta2 / 24.0) * np.dot(Sr, Sr)
    return np.mat(R)

In [164]:
def camera_full_projection_matrix(width, height, zoom, skew, pan, tilt, roll, Tx, Ty, Tz):
    """
    Creates projection matrix P from camera model parameters
    :param width: camera image width
    :param height: camera image height
    :param zoom: camera focal
    :param skew:
    :param pan:
    :param tilt:
    :param roll:
    :param Tx:
    :param Ty:
    :param Tz:
    :return: the projection Matrix P
    """
    aspect_ratio = float(width)/float(height)
    K = np.array([[zoom, skew, 0.5], [0.0, zoom * aspect_ratio, 0.5], [0.0, 0.0, 1.0]])

    # Rotation matrix
    Rpan = rodrigues(np.array([0.0, 0.0, pan*math.pi/180.0]))
    Rtilt = rodrigues(np.array([tilt*math.pi/180.0, 0.0, 0.0]))
    Rroll = rodrigues(np.array([0.0, 0.0, roll*math.pi/180.0]))
    Mrot = Rroll * Rtilt * Rpan

    # Translation vector
    t = np.array([Tx, Ty, Tz])
    KR = K * Mrot
    Kt = np.dot(K, t)

    # Projection Martix P
    P = np.zeros((3, 4))
    P[:, 0] = KR[:, 0].T
    P[:, 1] = KR[:, 1].T
    P[:, 2] = KR[:, 2].T
    P[:, 3] = Kt
    return P

In [165]:
def world_to_texture(pw, P):
    """
    Projects a world point to texture projection plane
    :param pw: world point (x, y, z)
    :param P: projection matrix P
    :return: texture point (i, j)
    """
    pw_h = np.append(pw, 1.0)
    pp = P.dot(pw_h)
    return np.array([pp[0]/pp[2], pp[1]/pp[2]])

In [166]:
def homography_pitch_to_texture(P):
    """
    Returns the homografy to transform points from the pitch (world Z=0) to texture image
    :param matrixP: Full matrix P
    :return: homography_pitch_to_texture matrix
    """
    # delete the 3th column, the z component
    return np.delete(P, 2, axis=1) 

In [167]:
def pitch_to_texture(pitch, P):
    """
    Project a point from pitch plane to texture plane
    :param pitch: pitch point (x, y)
    :param P: projection matrix P
    :return: texture point (i, j)
    """
    pitch_h = np.append(pitch, 1.0)
    H = homography_pitch_to_texture(P)
    transformed = H.dot(pitch_h)
    #print(projected)
    return np.array([transformed[0]/transformed[2], transformed[1]/transformed[2]])
    

In [168]:
def texture_to_pitch(texture, P):
    """
    Project a point from texture plane to pitch plane
    :param texture: texture point (i, j)
    :param P: projection matrix P
    :return: pitch point (x, y)
    """
    texture_h = np.append(texture, 1.0)
    H = homography_pitch_to_texture(P)
    Hinv = np.linalg.inv(H)
    transformed = Hinv.dot(texture_h)
    return np.array([transformed[0]/transformed[2], transformed[1]/transformed[2]])

## Testing

In [169]:
# test
# test matrixP_from_camera_model
camera_model = {
    "width": 5760,
    "height": 1080,
    "zoom": 0.45361389,
    "skew": 0.0,
    "pan": -2.0953152,
    "tilt": 108.76381,
    "roll": 0.0,
    "Tx": 0.48063888,
    "Ty": -0.30635475,
    "Tz": 87.349004}
P = camera_full_projection_matrix(**camera_model)
P_expected = np.array([[0.436, 0.4897, -0.1608, 43.893],
                      [0.01114, -0.3046, -2.4515, 42.933],
                      [-0.03462, 0.9462, -0.3217, 87.349]])
assert(np.allclose(P, P_expected, rtol=1e-3))

# ML
camera_model = {
    "width": 1920,
    "height": 1080,
    "zoom": 1.2124214,
    "skew": 0.0,
    "pan": -28.826538,
    "tilt": 110.37401,
    "roll": -10.530287,
    "Tx": 34.07756,
    "Ty": -3.4855517,
    "Tz": 74.498503}
P = camera_full_projection_matrix(**camera_model)
print(P)
P_expected = np.array([[0.8555, 0.9178, -0.3818, 78.566],
                      [-0.2154, -0.4256, -2.1606, 29.736],
                      [-0.452, 0.8213, -0.3481, 74.499]])
assert(np.allclose(P, P_expected, rtol=1e-3))

print()
# project world point
x = -10.0
y = 3.0
world = np.array([x, y, 0.0])
print("world: {}".format(world))
texture = world_to_texture(world, P)
print("texture: {}".format(texture))
"""
if (texture > 1.0).any() or (texture < 0.0).any():
    print("point is out the texture limits")
else:
    print("point is in the texture limits")
"""

print()
# transform point from pitch to texture plane
pitch = np.array([x, y])
print("pitch: {}".format(pitch))
texture = pitch_to_texture(pitch, P)
print("texture: {}".format(texture))

print()
# transform point from texture to pitch
print("texture: {}".format(texture))
pitch = texture_to_pitch(texture, P)
print("pitch: {}".format(pitch))



[[  0.85549003   0.91779104  -0.38178799  78.5656145 ]
 [ -0.21537938  -0.42563355  -2.16061685  29.73643822]
 [ -0.45199561   0.82127568  -0.34814685  74.498503  ]]

world: [-10.   3.   0.]
texture: [ 0.89300498  0.37570536]

pitch: [-10.   3.]
texture: [ 0.89300498  0.37570536]

texture: [ 0.89300498  0.37570536]
pitch: [-10.   3.]


# End

In [171]:
# test matrixP_from_camera_model
camera_model = {
    "width": 1920,
    "height": 1080,
    "zoom": 1.1870557,
    "skew": 0.0,
    "pan": -29.295381,
    "tilt": 110.26088,
    "roll": -10.845953,
    "Tx": 33.974704,
    "Ty": -3.5828012,
    "Tz": 72.705141}
P = camera_full_projection_matrix(**camera_model)
print(P)

[[  0.8250794    0.9120803   -0.38269407  76.68243654]
 [ -0.22463131  -0.41117778  -2.11752907  28.79170901]
 [ -0.45903625   0.81814751  -0.34629521  72.705141  ]]


In [172]:
# test matrixP_from_camera_model
camera_model = {
    "width": 1920,
    "height": 1080,
    "zoom": 1.343025,
    "skew": 0.0,
    "pan": -1.6334474,
    "tilt": 113.66455,
    "roll": 0.52238362,
    "Tx": -0.89986081,
    "Ty": -4.878311,
    "Tz": 80.464399}
P = camera_full_projection_matrix(**camera_model)
print(P)

[[  1.32922926e+00   5.00963792e-01  -1.89475605e-01   3.90236639e+01]
 [  3.60216954e-02  -4.99518453e-01  -2.38742906e+00   2.85847442e+01]
 [ -2.61082052e-02   9.15538926e-01  -4.01381162e-01   8.04643990e+01]]


In [173]:
# test matrixP_from_camera_model
camera_model = {
    "width": 1920,
    "height": 1080,
    "zoom": 1.1721035,
    "skew": 0.0,
    "pan": 24.789001,
    "tilt": 110.79087,
    "roll": 10.340134,
    "Tx": -34.08269,
    "Ty": -3.9479977,
    "Tz": 73.438364}
P = camera_full_projection_matrix(**camera_model)
print(P)

[[  1.27412007e+00   8.71037377e-03   1.92038151e-02  -3.22925824e+00]
 [  2.30460347e-01  -3.93028116e-01  -2.09389290e+00   2.84925830e+01]
 [  3.91975382e-01   8.48740315e-01  -3.54957995e-01   7.34383640e+01]]
